In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import xarray as xr
import fsspec
import plotly.express as px
import data_model as dm

In [2]:
metadata = pd.read_feather('data\\metadata.fea')
time_series = pd.read_feather('data\\timeseries.fea')
time_series_smoothed = pd.read_feather('data\\timeseries_smoothed.fea')

In [3]:
stations = [int(x) for x in time_series.columns[1:].tolist()]

In [2]:
new_model = dm.data_model()

In [3]:
x_meta, x_series, y = new_model.train()

In [6]:
x_series[0][63]

0.052351017

In [75]:
metadata

,index,ss_id,latitude_rounded,longitude_rounded,llsoacd,orientation,tilt,kwp,operational_at
0,198,2603,51.70,-4.71,W01000610,210.0,32.0,3.96,2012-04-02
1,202,2607,52.44,-0.12,E01018211,200.0,35.0,2.94,2011-08-25
2,220,2625,50.38,-4.22,E01033294,190.0,38.0,2.35,2011-08-23
3,221,2626,54.99,-3.18,S00097762,270.0,22.0,3.60,2011-08-18
4,223,2628,50.99,-1.37,E01022706,225.0,34.0,1.38,2011-09-20
...,...,...,...,...,...,...,...,...,...
1305,24657,27063,51.41,-2.49,E01014398,185.0,35.0,4.00,2018-04-18
1306,24658,27064,51.47,-0.59,E01016588,180.0,45.0,2.00,2018-04-18
1307,24659,27065,51.36,-2.93,E01014817,125.0,37.0,4.00,2018-04-18
1308,24660,27066,51.44,-2.85,E01014759,165.0,35.0,4.00,2018-04-18


In [4]:
px.scatter_geo(metadata,lat='latitude_rounded',lon='longitude_rounded', hover_name="ss_id")

In [96]:
def distance(df):
    x = df['latitude_rounded'].to_numpy()
    y = df['longitude_rounded'].to_numpy()
    x_distance = np.subtract(x, 55.8642)
    y_distance = np.subtract(y, -4.2518)
    x_2 = np.power(x_distance,2)
    y_2 = np.power(y_distance,2)
    sum = np.add(x_2,y_2)
    distance = np.power(sum,1/2)
    return distance

In [97]:
distances = distance(metadata)

In [69]:
metadata['distance'].min()

4.154728015165371

In [88]:
metadata['latitude_rounded'].to_numpy()

array([51.7 , 52.44, 50.38, ..., 51.36, 51.44, 53.52])

In [98]:
metadata['distance'] = distances.tolist()

In [184]:
str(metadata.iloc[[1]]['ss_id'].iat[0])

'2607'

In [132]:
cropped = metadata[(metadata['distance']<=0.977)]

In [146]:
cropped.to_feather('train.fea')

In [152]:
stationscropped = [str(x) for x in cropped['ss_id'].tolist()]

In [159]:
croppedtimeseries = time_series[stationscropped]

In [162]:
croppedtimeseries

,datetime,6433,6330,6478,6425,10702,3231,6054,4030,14531,...,5471,11287,3125,6619,7720,3865,10648,16920,7758,2760
0,2018-01-01 08:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0
1,2018-01-01 08:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0
2,2018-01-01 08:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0
3,2018-01-01 08:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0
4,2018-01-01 08:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200180,2021-10-27 19:35:00,0.0,0.0,0.0,0.0,0.0,0.0,0.454061,0.0,0.0,...,0.0,0.0,0.011794,0.026584,0.623306,0.002509,0.0,0.055036,0.36434,0.0
200181,2021-10-27 19:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.454061,0.0,0.0,...,0.0,0.0,0.011794,0.026584,0.623306,0.002509,0.0,0.055036,0.36434,0.0
200182,2021-10-27 19:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.454061,0.0,0.0,...,0.0,0.0,0.011794,0.026584,0.623306,0.002509,0.0,0.055036,0.36434,0.0
200183,2021-10-27 19:50:00,0.0,0.0,0.0,0.0,0.0,0.0,0.454061,0.0,0.0,...,0.0,0.0,0.011794,0.026584,0.623306,0.002509,0.0,0.055036,0.36434,0.0


In [176]:
croppedtimeseries.iloc[[2]].iloc[:,2]

2    0.0
Name: 6330, dtype: float32

In [147]:
test_set = cropped.head(85)

In [149]:
test_set.to_feather('test.fea')

In [144]:
px.scatter_geo(cropped,lat='latitude_rounded',lon='longitude_rounded', hover_name="ss_id")

In [ ]:
(time_series.isna().values.any() or time_series_smoothed.isna().values.any())

In [ ]:
with fsspec.open('data\\passiv.netcdf', mode="rb") as file:
    dataset = xr.load_dataset(file, engine="h5netcdf")
df = dataset.to_dataframe()

In [ ]:
collist = time_series.columns.tolist()
collist = collist[1:]

In [ ]:
time_series_smoothed = time_series_smoothed.dropna(axis=1, how='all')

In [ ]:
time_series[collist] = time_series[collist].interpolate(method ='linear', limit_direction ='backward', limit = 1000000)

In [ ]:
df_max_scaled = df.copy()

In [ ]:
for column in df_max_scaled.columns:
    df_max_scaled[column] = df_max_scaled[column]  / df_max_scaled[column].abs().max()

In [ ]:
start = datetime.time(8,00,00)

In [ ]:
df['time'] = df.datetime.apply(lambda x: x.time())

In [ ]:
start = datetime.strptime("08:00:00", "%H:%M:%S").time()
end = datetime.strptime("20:00:00", "%H:%M:%S").time()

In [ ]:

mask = (df.time<end) & (df.time>=start)
df = df.loc[mask]

In [ ]:
df

In [ ]:
df.to_feather('timeseries.fea')

In [ ]:
collist = df.columns.tolist()
collist = collist[1:]

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.lineplot(
    x='datetime',
    y='10003',
    data= time_series[0:1000])
plt.show()

In [ ]:
smoothed = df.copy()

In [ ]:
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


In [ ]:
fs = 5000.0
lowcut = 500.0
highcut = 1250.0

In [ ]:
smoothed[collist] = butter_bandpass_filter(df[collist], lowcut, highcut, fs, order=6)

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.lineplot(
    x='datetime',
    y='10003',
    data=smoothed[25:75])
plt.show()

In [ ]:
time_series_smoothed.to_feather('timeseries_smoothed.fea')